#단어를 토큰화

In [2]:
from nltk.tokenize import word_tokenize
import random

docs = ["I am programming Python",
       "Economics is a study of money",
       "Python is easy",
       "I like money",
       "i like Economics and study",
       "I like easy programming Economics is money"]
print(type(docs))

        
data = [[word.lower() for word in word_tokenize(doc)] for doc in docs]
print(data)


<class 'list'>
[['i', 'am', 'programming', 'python'], ['economics', 'is', 'a', 'study', 'of', 'money'], ['python', 'is', 'easy'], ['i', 'like', 'money'], ['i', 'like', 'economics', 'and', 'study'], ['i', 'like', 'easy', 'programming', 'economics', 'is', 'money']]


단어에서 stopwords제거-문서별로 단어가 추출되어있다

In [44]:
from spacy.lang.en.stop_words import STOP_WORDS

def remove_stopwords(docs):
    return [[word for word in doc if word not in STOP_WORDS] for doc in docs]

data = remove_stopwords(data)
print (data)



[['programming', 'python'], ['economics', 'study', 'money'], ['python', 'easy'], ['like', 'money'], ['like', 'economics', 'study'], ['like', 'easy', 'programming', 'economics', 'money']]


이젠 문서들을 한덩어리로 보고 전체에서 단어를 추출한다 / 단어에 숫자메기고(dictionary) 그걸 컴이 알아듣게 숫자로 바꿈(corpus)

In [45]:
import gensim.corpora as corpora
#숫자 부여하기
id2word = corpora.Dictionary(data)
#이런식으로 부여되어있다 사람한테 보여줌
dic = [id2word[i] for i in range (len(id2word))]

print(dic)
print([(i, id2word[i]) for i in range(len(id2word))])




##컴이 알아듣게 숫자로 바꿈-이건 아직 문서들이 나눠져있음
corpus = [id2word.doc2bow(doc) for doc in data]

print(corpus)

['programming', 'python', 'economics', 'money', 'study', 'easy', 'like']
[(0, 'programming'), (1, 'python'), (2, 'economics'), (3, 'money'), (4, 'study'), (5, 'easy'), (6, 'like')]
[[(0, 1), (1, 1)], [(2, 1), (3, 1), (4, 1)], [(1, 1), (5, 1)], [(3, 1), (6, 1)], [(2, 1), (4, 1), (6, 1)], [(0, 1), (2, 1), (3, 1), (5, 1), (6, 1)]]


LDA_model-앞서 만든 corpus와 id2word를 가지고 컴퓨터가 토픽을 2개(내가정함)생성한다

In [46]:
import gensim

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10, per_word_topics=True)
print(lda_model)

LdaModel(num_terms=7, num_topics=2, decay=0.5, chunksize=2000)


In [47]:
#토픽 2개의 구성은 각각 이렇다
from pprint import pprint
pprint(lda_model.show_topics(num_topics=2))

[(0,
  '0.236*"economics" + 0.214*"money" + 0.213*"like" + 0.201*"study" + '
  '0.046*"python" + 0.045*"programming" + 0.044*"easy"'),
 (1,
  '0.210*"easy" + 0.209*"programming" + 0.208*"python" + 0.117*"like" + '
  '0.117*"money" + 0.094*"economics" + 0.044*"study"')]


생성한 토픽 2개와 7개의 문서가 각각 얼만큼 관계있는지 알려줌

In [48]:

for i in range(len(corpus)):
    topic0, topic1 = lda_model.get_document_topics(corpus[i])
    print('{0:f},{1:f}, {2:s}'.format(topic0[1], topic1[1], docs[i]))

0.171145,0.828855, I am programming Python
0.863429,0.136571, Economics is a study of money
0.171028,0.828972, Python is easy
0.802594,0.197406, I like money
0.863308,0.136692, i like Economics and study
0.422543,0.577457, I like easy programming Economics is money


아까 전체문서에서 뽑은 단어 7개(dic)가 각 토픽과 얼만큼 관계있는지 보여줌


In [49]:
for i in range(len(dic)):
    topic0, topic1 =lda_model.get_term_topics(dic[i])
    print('{0:f},{1:f}, {2:s}'.format(topic0[1], topic1[1], dic[i]))
    

0.015414,0.175820, programming
0.016245,0.174661, python
0.204660,0.057997, economics
0.182193,0.080850, money
0.169366,0.013435, study
0.014716,0.176808, easy
0.181594,0.081467, like


# 이거뭐냐

In [50]:
lda_model.get_document_topics(corpus[0], per_word_topics=True)

([(0, 0.17114735), (1, 0.82885265)],
 [(0, [1]), (1, [1])],
 [(0, [(1, 0.9934996)]), (1, [(1, 0.9931066)])])

각 단어가 2개의 토픽과 얼만큼 관련있는지. Programming이란 단어는 토픽 1과 0.993만큼 관련 있음 / Economics란 단어는 토픽 0과 토픽 1 둘 다와 관련이 있다. 같은단어라도 서로 다른 문서에 속해 있다면 주어진 2개의 토픽과 연관률이 다를 수 있다

In [54]:
print(data) #data는 문서별로 나눠져있던 것

for i in range(len(data)):
    word_topics = lda_model.get_document_topics(corpus[i], per_word_topics=True)
    for j in range(len(data[i])):
        topic = word_topics[2][j][1]
        print(data[i][j], topic)

[['programming', 'python'], ['economics', 'study', 'money'], ['python', 'easy'], ['like', 'money'], ['like', 'economics', 'study'], ['like', 'easy', 'programming', 'economics', 'money']]
programming [(1, 0.9935011)]
python [(1, 0.9931083)]
economics [(0, 0.9837064), (1, 0.016293552)]
study [(0, 0.9747184), (1, 0.025281657)]
money [(0, 0.9953848)]
python [(1, 0.9931186)]
easy [(1, 0.9938371)]
like [(0, 0.9543172), (1, 0.045682725)]
money [(0, 0.9538398), (1, 0.046160113)]
like [(0, 0.9836767), (1, 0.016323367)]
economics [(0, 0.99537617)]
study [(0, 0.9744021), (1, 0.025597872)]
like [(0, 0.05704304), (1, 0.942957)]
easy [(0, 0.7088725), (1, 0.29112744)]
programming [(0, 0.6085987), (1, 0.39140126)]
economics [(0, 0.054312736), (1, 0.94568723)]
money [(0, 0.6060002), (1, 0.39399984)]
